In [1]:
import glob
import os

import cv2
from loguru import logger as log



In [37]:
import pandas as pd

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
DDSM_ROOT_DIR = os.path.join(os.getenv('data_path'),'datasets','ddsm_ds')

In [33]:
file_paths = []
for root, case, files in os.walk(f"{DDSM_ROOT_DIR}/DDSM_JPEG"):
    for file in files:
        if file.endswith(".png"):
            path = os.path.join(root, file)
            file_paths.append(path)

In [58]:
file_paths.__len__()

9854

In [40]:
file_paths[:2]

['/Users/krystianzielinski/Documents/repos/federated_learning_demo/data/datasets/ddsm_ds/DDSM_JPEG/normal_12/case2039/A_2039_1.LEFT_CC.LJPEG.png',
 '/Users/krystianzielinski/Documents/repos/federated_learning_demo/data/datasets/ddsm_ds/DDSM_JPEG/normal_12/case2039/A_2039_1.RIGHT_MLO.LJPEG.png']

In [41]:
num_img_per_case = [x.split('/')[-3:] for x in file_paths]

In [44]:
num_img_per_case_df = pd.DataFrame(num_img_per_case)

In [46]:
num_img_per_case_df['patient_id'] = num_img_per_case_df[0]+num_img_per_case_df[1]

In [49]:
num_img_per_case_df

,0,1,2,patient_id
0,normal_12,case2039,A_2039_1.LEFT_CC.LJPEG.png,normal_12case2039
1,normal_12,case2039,A_2039_1.RIGHT_MLO.LJPEG.png,normal_12case2039
2,normal_12,case2039,A_2039_1.LEFT_MLO.LJPEG.png,normal_12case2039
3,normal_12,case2039,A_2039_1.RIGHT_CC.LJPEG.png,normal_12case2039
4,normal_12,case2030,A_2030_1.RIGHT_CC.LJPEG.png,normal_12case2030
...,...,...,...,...
9849,normal_02,case0313,A_0313_1.RIGHT_MLO.LJPEG.png,normal_02case0313
9850,normal_02,case0314,A_0314_1.LEFT_CC.LJPEG.png,normal_02case0314
9851,normal_02,case0314,A_0314_1.RIGHT_MLO.LJPEG.png,normal_02case0314
9852,normal_02,case0314,A_0314_1.RIGHT_CC.LJPEG.png,normal_02case0314


In [56]:
counter = num_img_per_case_df.groupby('patient_id').count()[2]

In [57]:
counter.loc[counter>4]

patient_id
cancer_01case3007    5
cancer_13case4140    5
Name: 2, dtype: int64

In [9]:
def ddsm_data_generator(read_img_as_8_bits: bool = False) -> dict:
    for root, _, files in os.walk(f"{DDSM_ROOT_DIR}/DDSM_JPEG"):
        for file in files:
            if file.endswith(".png"):
                path = os.path.join(root, file)
                # if read_img_as_8_bits:
                #     img = cv2.imread(path)
                # else:
                #     img = cv2.imread(path, cv2.IMREAD_UNCHANGED)
                file_description = path.split("/")[-1].replace(".LJPEG.png", "").split('.')
                density = int(file_description[0].split('_')[0])
                patient_id = file_description[0].split('_')[1]
                which_brest = file_description[1].split('_')[0]
                image_view = file_description[1].split('_')[1]
                yield {
                    # "img": img,
                    "density": density,
                    "which_breast": which_brest,
                    "image_view": image_view,
                    "patient_id": patient_id
                }


# generator = ddsm_data_generator(read_img_as_8_bits=False)
# print(next(generator))
# print(next(generator)["img"].shape)
# print(next(generator)["img"])


In [10]:
ddsm_data_generator()

<generator object ddsm_data_generator at 0x1385ef270>

In [ ]:
import os

import cv2
from loguru import logger as log

from data_generators import consts
from data_generators.data_augmentation import data_augmentation
from datasets.ddsm_ds.ddsm_data_generator_kz import ddsm_data_generator
from general_tools.patching import find_patch

TEST_DATA_SPLIT = 0.2
TARGET_IMG_SIZE = (224, 224)
PATCHING = False

MIAS_DIR_MAP_TRAIN = {
    "F": consts.TWO_CLASSES_TRAIN_NOT_DENSE_PATH,
    "G": consts.TWO_CLASSES_TRAIN_NOT_DENSE_PATH,
    "D": consts.TWO_CLASSES_TRAIN_DENSE_PATH
}
OTHER_DIR_MAP_TRAIN = {
    1: consts.TWO_CLASSES_TRAIN_NOT_DENSE_PATH,
    2: consts.TWO_CLASSES_TRAIN_NOT_DENSE_PATH,
    3: consts.TWO_CLASSES_TRAIN_DENSE_PATH,
    4: consts.TWO_CLASSES_TRAIN_DENSE_PATH
}

MIAS_DIR_MAP_TEST = {
    "F": consts.TWO_CLASSES_TEST_NOT_DENSE_PATH,
    "G": consts.TWO_CLASSES_TEST_NOT_DENSE_PATH,
    "D": consts.TWO_CLASSES_TEST_DENSE_PATH
}
OTHER_DIR_MAP_TEST = {
    1: consts.TWO_CLASSES_TEST_NOT_DENSE_PATH,
    2: consts.TWO_CLASSES_TEST_NOT_DENSE_PATH,
    3: consts.TWO_CLASSES_TEST_DENSE_PATH,
    4: consts.TWO_CLASSES_TEST_DENSE_PATH
}


def id_generator():
    id = 0
    while True:
        yield id
        id += 1


def convert_16b_to_8b(img_16b):
    return (img_16b / 256).astype('uint8')


def generate_data_for_given_generator(data_generator, id_generator, regularization_factor, test_map, train_map,
                                      additional_prepro_steps=None, augment=False):
    test_count = int(1 / TEST_DATA_SPLIT)
    counter = 1

    nd_counter = 0
    d_counter = 0

    images_counter = 1
    for data in data_generator:
        if not data["img"]:
            log.warning(f"No image found for for {data_generator}")
            continue

        try:
            if counter == test_count:
                target_dir = test_map[data["density"]]
                counter = 1
            else:
                target_dir = train_map[data["density"]]
                counter += 1
        except KeyError:
            continue
        # data["img"] = data["img"] / regularization_factor
        if additional_prepro_steps:
            data["img"] = additional_prepro_steps(data["img"])
        if PATCHING:
            data["img"] = find_patch(data["img"])
        data["img"] = cv2.resize(data["img"], TARGET_IMG_SIZE)
        file_name = os.path.join(target_dir, f"{next(id_generator)}.png")
        cv2.imwrite(file_name, data["img"])
        if data["density"] in [3, 4] or data["density"] == "D":
            d_counter += 1
            if augment:
                augmented_image = data_augmentation(data["img"])
                augmented_file_name = os.path.join(target_dir, f"{next(id_generator)}.png")
                cv2.imwrite(augmented_file_name, augmented_image)
        else:
            nd_counter += 1

        if images_counter % 100 == 0:
            log.info(f"Parsed another 100 images")
        images_counter += 1
    print("DENSE: ", d_counter)
    print("NOT DENSE: ", nd_counter)


def generate_dataset_two_classes():
    ddsm = ddsm_data_generator()

    id = id_generator()

    log.info("Start parsing DDSM")
    generate_data_for_given_generator(ddsm, id, consts.REGULARIZATION_FACTOR_16_BIT, OTHER_DIR_MAP_TEST,
                                      OTHER_DIR_MAP_TRAIN,
                                      additional_prepro_steps=convert_16b_to_8b, augment=False)
    log.info("DDSM parsed")

if __name__ == "__main__":
    generate_dataset_two_classes()
